# CHI 660F 电化学工作站测试

本 Notebook 用于快速测试 5 种电化学技术：**CV / LSV / i-t / EIS / OCPT**

### 使用方法
1. 确保 `D:\CHI660F\chi660f.exe` 已安装
2. 按需修改各 Cell 中的参数
3. 设置 `dummy=True` 使用 Dummy Cell 测试，`dummy=False` 连接实际电极
4. 依次运行对应技术的 Cell

---

In [ ]:
# ===== 公共导入 & 辅助函数 =====
import sys, os
sys.path.insert(0, os.getcwd())  # 确保能找到 src 包

from src.models import ECSettings, ECTechnique
from src.echem_sdl.hardware.chi_echem_bridge import run_echem, close_echem

def show_result(result):
    """打印实验结果摘要"""
    print(f"成功: {result.success}")
    print(f"技术: {result.technique}")
    print(f"数据点: {len(result.data_points)}")
    print(f"耗时: {result.elapsed_time:.1f}s")
    if result.data_file:
        print(f"数据文件: {result.data_file}")
    if result.error_message:
        print(f"错误: {result.error_message}")
    if result.headers:
        print(f"列名: {result.headers}")
    if result.data_points:
        print(f"前3行: {result.data_points[:3]}")

print("导入完成 ✅")

: 

---
## 1. CV — 循环伏安法

| 参数 | 说明 | 典型值 |
|------|------|--------|
| `e0` | 初始电位 (V) | 0 |
| `eh` | 上限电位 (V) | 0.8 ~ 1.5 |
| `el` | 下限电位 (V) | -0.5 ~ -0.2 |
| `ef` | 终止电位 (V) | 0 |
| `scan_rate` | 扫描速率 (V/s) | 0.05 ~ 0.1 |
| `seg_num` | 扫描段数 | 2 (1圈) |
| `sample_interval_ms` | 采样间隔 (mV) | 1 |
| `sensitivity` | 灵敏度 (A/V) | 1e-1 ~ 1e-6 |
| `quiet_time_s` | 静置时间 (s) | 2 |

In [ ]:
# ===== CV 循环伏安法 =====
result_cv = run_echem(
    ECSettings(
        technique=ECTechnique.CV,
        e0=0,              # 初始电位 (V)
        eh=1.5,            # 上限电位 (V)
        el=-0.5,           # 下限电位 (V)
        ef=0,              # 终止电位 (V)
        scan_rate=0.1,     # 扫描速率 (V/s)
        seg_num=2,         # 扫描段数 (2段=1圈)
        sample_interval_ms=1,  # 采样间隔 (mV)
        sensitivity=1e-1,  # 灵敏度 (A/V)
        quiet_time_s=2,    # 静置时间 (s)
    ),
    dummy=True,  # True=Dummy Cell测试, False=实际测量
)
show_result(result_cv)

---
## 2. LSV — 线性扫描伏安法

| 参数 | 说明 | 典型值 |
|------|------|--------|
| `e0` | 初始电位 (V) | -0.5 |
| `ef` | 终止电位 (V) | 1.0 |
| `scan_rate` | 扫描速率 (V/s) | 0.05 |
| `sample_interval_ms` | 采样间隔 (mV) | 1 |
| `sensitivity` | 灵敏度 (A/V) | 1e-1 |
| `quiet_time_s` | 静置时间 (s) | 2 |

In [ ]:
# ===== LSV 线性扫描伏安法 =====
result_lsv = run_echem(
    ECSettings(
        technique=ECTechnique.LSV,
        e0=-0.5,           # 初始电位 (V)
        ef=1.0,            # 终止电位 (V)
        scan_rate=0.05,    # 扫描速率 (V/s)
        sample_interval_ms=1,  # 采样间隔 (mV)
        sensitivity=1e-1,  # 灵敏度 (A/V)
        quiet_time_s=2,    # 静置时间 (s)
    ),
    dummy=True,
)
show_result(result_lsv)

---
## 3. i-t — 计时电流法 (Amperometric i-t)

| 参数 | 说明 | 典型值 |
|------|------|--------|
| `e0` | 恒电位 (V) | 0.2 |
| `sample_interval_ms` | 采样间隔 (ms) | 100 |
| `run_time_s` | 运行时间 (s) | 30 ~ 300 |
| `sensitivity` | 灵敏度 (A/V) | 1e-4 |
| `quiet_time_s` | 静置时间 (s) | 2 |

In [ ]:
# ===== i-t 计时电流法 =====
result_it = run_echem(
    ECSettings(
        technique=ECTechnique.I_T,
        e0=0.2,              # 恒电位 (V)
        sample_interval_ms=100,  # 采样间隔 (ms)
        run_time_s=30,       # 运行时间 (s)
        sensitivity=1e-4,    # 灵敏度 (A/V)
        quiet_time_s=2,      # 静置时间 (s)
    ),
    dummy=True,
)
show_result(result_it)

---
## 4. EIS — 交流阻抗谱 (Electrochemical Impedance Spectroscopy)

| 参数 | 说明 | 典型值 |
|------|------|--------|
| `e0` | 偏置电位 (V) | 0 |
| `freq_high` | 最高频率 (Hz) | 100000 |
| `freq_low` | 最低频率 (Hz) | 1 |
| `amplitude` | AC 振幅 (V) | 0.005 |
| `bias_mode` | 偏置模式 (0=vs Eref, 1=vs Eoc) | 0 |
| `sensitivity` | 灵敏度 (A/V) | 1e-3 |
| `quiet_time_s` | 静置时间 (s) | 2 |

In [ ]:
# ===== EIS 交流阻抗谱 =====
result_eis = run_echem(
    ECSettings(
        technique=ECTechnique.EIS,
        e0=0,                # 偏置电位 (V)
        freq_high=100000,    # 最高频率 (Hz)
        freq_low=1,          # 最低频率 (Hz)
        amplitude=0.005,     # AC振幅 (V)
        bias_mode=0,         # 0=vs Eref, 1=vs Eoc
        sensitivity=1e-3,    # 灵敏度 (A/V)
        quiet_time_s=2,      # 静置时间 (s)
    ),
    dummy=True,
)
show_result(result_eis)

---
## 5. OCPT — 开路电位计时法 (Open Circuit Potential vs Time)

| 参数 | 说明 | 典型值 |
|------|------|--------|
| `sample_interval_ms` | 采样间隔 (ms) | 1000 |
| `run_time_s` | 运行时间 (s) | 60 ~ 300 |
| `eh` | 上限电位 (V) | 10 |
| `el` | 下限电位 (V) | -10 |

> OCPT 不需要设置灵敏度，测量的是开路电位。

In [ ]:
# ===== OCPT 开路电位 =====
result_ocpt = run_echem(
    ECSettings(
        technique=ECTechnique.OCPT,
        sample_interval_ms=1000,  # 采样间隔 (ms)
        run_time_s=60,       # 运行时间 (s)
        eh=10,               # 上限电位 (V)
        el=-10,              # 下限电位 (V)
    ),
    dummy=True,
)
show_result(result_ocpt)

---
## 清理：关闭 CHI 连接

测试完毕后运行此 Cell 关闭连接。

In [ ]:
# ===== 关闭连接 =====
close_echem()
print("CHI 660F 连接已关闭 ✅")

---
## 附录：数据可视化

选择上面任意一个 `result_xxx`，运行下方 Cell 即可绘图。

In [ ]:
# ===== 数据可视化 (选择要绘制的 result) =====
import matplotlib.pyplot as plt
import numpy as np

# 👇 修改这里选择要绘制的结果
result = result_cv   # result_cv / result_lsv / result_it / result_eis / result_ocpt

if result.success and result.data_points:
    data = np.array(result.data_points)
    tech = result.technique.upper()
    
    fig, ax = plt.subplots(figsize=(8, 5))
    
    if tech == 'CV' and data.shape[1] >= 3:
        ax.plot(data[:, 1], data[:, 2])
        ax.set_xlabel('E / V')
        ax.set_ylabel('I / A')
        ax.set_title('Cyclic Voltammetry (CV)')
    elif tech == 'LSV' and data.shape[1] >= 3:
        ax.plot(data[:, 1], data[:, 2])
        ax.set_xlabel('E / V')
        ax.set_ylabel('I / A')
        ax.set_title('Linear Sweep Voltammetry (LSV)')
    elif tech in ('I-T', 'IT') and data.shape[1] >= 3:
        ax.plot(data[:, 0], data[:, 2])
        ax.set_xlabel('t / s')
        ax.set_ylabel('I / A')
        ax.set_title('Amperometric i-t Curve')
    elif tech == 'EIS' and data.shape[1] >= 2:
        ax.plot(data[:, 0], -data[:, 1], 'o-', markersize=3)
        ax.set_xlabel("Z' / Ω")
        ax.set_ylabel("-Z'' / Ω")
        ax.set_title('Nyquist Plot (EIS)')
        ax.set_aspect('equal')
    elif tech == 'OCPT' and data.shape[1] >= 2:
        ax.plot(data[:, 0], data[:, 1])
        ax.set_xlabel('t / s')
        ax.set_ylabel('E / V')
        ax.set_title('Open Circuit Potential vs Time')
    else:
        ax.plot(data[:, 0], data[:, 1] if data.shape[1] > 1 else data[:, 0])
        ax.set_title(tech)
    
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    print(f"数据点数: {len(data)}, 列: {result.headers}")
else:
    print(f"无数据可绘制: {result.error_message}")